In [1]:
import os
os.chdir("..")
print(os.getcwd())

/Users/andreaspletschko/Uni/5. Semester (HKUST)/Machine_Learning/COMP5212


In [17]:
MODEL_PATH = "checkpoints/model_ncg02.hpc.itc.rwth-aachen.de_20231122-073926_20_epochs_.pth"
TSV_PATH = "data/kaggle_dataset/train.tsv"
IMAGE_PATH = "notebooks/demo_images/1.png"

In [3]:
from PIL import Image
import torch
from torchvision import transforms

/Users/andreaspletschko/.local/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
from src.models.model import get_model

In [5]:
def load_model(model_path):
    state_dict = torch.load(model_path, map_location=torch.device('cpu'))
    model = get_model(model_name="mobilenet_v3_small", num_classes=56)
    model.load_state_dict(state_dict)
    print(model)
    model.eval()
    return model

def get_idx_to_label(tsv_path):
    labels = []
    tsv_file = open(tsv_path, "r")

    for line in tsv_file:
        if line in ['\n', '\r\n']:
            continue
        label, _ = os.path.split(line.strip())
        labels.append(label)

    idx_to_label = {idx: label for idx, label in enumerate(set(labels))}
    return idx_to_label

def load_img(image_path):
    transform = transforms.Compose([
        transforms.Resize((1536, 662)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    image = Image.open(image_path).convert("RGB")
    image_transformed = transform(image)
    return image_transformed

In [18]:
print(os.getcwd())
idx_to_label = get_idx_to_label(TSV_PATH)
image = load_img(IMAGE_PATH)
image = image.unsqueeze(0)

model = load_model(MODEL_PATH)

with torch.no_grad():
    output = model(image)
    # top 3
    _, predicted = torch.topk(output.data, 3, dim=1)
    predicted = predicted.squeeze(0)
    print(predicted)
    print(idx_to_label[predicted[0].item()])
    print(idx_to_label[predicted[1].item()])
    print(idx_to_label[predicted[2].item()])


/Users/andreaspletschko/Uni/5. Semester (HKUST)/Machine_Learning/COMP5212
MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2)